### Fine-tuning MobileNet to recognize the 'Avengers'

Let's load the MobileNet model first, and make the necessary imports

In [9]:
import numpy as np
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import imagenet_utils

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import os 
import random 
import matplotlib.pyplot as plt
%matplotlib inline 

In [10]:
# GPU configuration 
# This code checks for available GPU devices and 
# sets memory growth to prevent TensorFlow from allocating all GPU memory at once.
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num GPUs Available:  0


Let's load / download the model

In [31]:
mobile = tf.keras.applications.mobilenet.MobileNet()

#### Preprocess the data

In [16]:
# make sure we are in the correct directory
# os.chdir('Documents/Deep-Learning-Chronicles/image_classification_CNN/')
os.getcwd()

'C:\\Users\\ianwr\\Documents\\Deep-Learning-Chronicles\\image_classification_CNN'

In [18]:
train_path = 'mobilenet_data/avengers/train/'
valid_path = 'mobilenet_data/avengers/valid/'
test_path = 'mobilenet_data/avengers/test/'

In [19]:
# create data generators
# why? 
# Data generators are used to efficiently load and preprocess batches of images during training, validation, and testing phases. 
#  the batch_size argument stands for the number of images to be processed in each batch during training or evaluation.
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory=train_path, target_size=(224,224), batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory=valid_path, target_size=(224,224), batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory=test_path, target_size=(224,224), batch_size=10, shuffle=False)

Found 274 images belonging to 5 classes.
Found 60 images belonging to 5 classes.
Found 60 images belonging to 5 classes.


In [22]:
assert train_batches.n == 274
assert valid_batches.n == 60
assert test_batches.n == 60

### Fine-tuning

In [32]:
mobile.summary()

Model: "mobilenet_1.00_224"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1 (Conv2D)                       │ (None, 112, 112, 32)        │             864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1_bn (BatchNormalization)        │ (None, 112, 112, 32)        │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1_relu (ReLU)                    │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_1 (DepthwiseConv2D)          │ (None, 112, 112, 32)        │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_1_bn (BatchNormalization)    │ (None, 112, 112, 32)        │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_1_relu (ReLU)                │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_1 (Conv2D)                   │ (None, 112, 112, 64)        │           2,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_1_bn (BatchNormalization)    │ (None, 112, 112, 64)        │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_1_relu (ReLU)                │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pad_2 (ZeroPadding2D)           │ (None, 113, 113, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_2 (DepthwiseConv2D)          │ (None, 56, 56, 64)          │             576 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_2_bn (BatchNormalization)    │ (None, 56, 56, 64)          │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_2_relu (ReLU)                │ (None, 56, 56, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_2 (Conv2D)                   │ (None, 56, 56, 128)         │           8,192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_2_bn (BatchNormalization)    │ (None, 56, 56, 128)         │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_2_relu (ReLU)                │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_3 (DepthwiseConv2D)          │ (None, 56, 56, 128)         │           1,152 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_3_bn (BatchNormalization)    │ (None, 56, 56, 128)         │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_3_relu (ReLU)                │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_3 (Conv2D)                   │ (None, 56, 56, 128)         │          16,384 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 4,253,864 (16.23 MB)

 Trainable params: 4,231,976 (16.14 MB)

 Non-trainable params: 21,888 (85.50 KB)

## Note:
Just leaving this friendly note here in case we have to retrace our steps, figuring out which layers to omit:

"the number of layers we are going to include or exclude from a pre-trained model when fine-tuning, is going to come through **experimentation** and **personal choice**"

In a past experiment, after deep research, we omitted upto the 6th last layer of the MobileNet model. 

# 
Let's try that again, if it doesn't work properly we can always retrace paths and make adjustments

In [33]:
# sixth layer from the end 
mobile.layers[-6]

<ReLU name=conv_pw_13_relu, built=True>

In [34]:
# print total layers
print("Total layers in the model: ", len(mobile.layers))

Total layers in the model:  91


In [35]:
# the output tensor of the sixth layer from the end of the MobileNet model. 
mobile.layers[-6].output

<KerasTensor shape=(None, 7, 7, 1024), dtype=float32, sparse=False, ragged=False, name=keras_tensor_360>

In [38]:
x = mobile.layers[-6].output

# Convert (None, 7, 7, 1024) -> (None, 1024)
x = GlobalAveragePooling2D()(x) 
# converts the 2D feature maps into a 1D feature vector by averaging each feature map. 
# why? to reduce dimensionality and retain important features 

# final output layer with softmax activation for multi-class classification
# 5 classes in the avengers dataset (ironman, captain america, hulk, thor, black widow) => (but with their real names) 
output = Dense(units=5, activation='softmax')(x)

In [39]:
output

<KerasTensor shape=(None, 5), dtype=float32, sparse=False, ragged=False, name=keras_tensor_369>

In [40]:
# define the new model
model = Model(inputs=mobile.input, outputs=output)
# how comes we are not using Sequential() here?
# Because we are building a model that combines layers from a pre-trained model (MobileNet) with new layers, 
# and the functional API allows for more flexibility in defining complex architectures compared to the Sequential API.

In [41]:
# freezing the first 23 layers 
for layer in model.layers[:23]:
    layer.trainable = False

# why are we freezing the first 23 layers?
# ---------------------------------------------
# helps retain the learned features from the pre-trained model, which are useful for 
# general image recognition tasks. By freezing these layers, we prevent their weights from being 
# updated during training on the new dataset.

In [42]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1 (Conv2D)                       │ (None, 112, 112, 32)        │             864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1_bn (BatchNormalization)        │ (None, 112, 112, 32)        │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1_relu (ReLU)                    │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_1 (DepthwiseConv2D)          │ (None, 112, 112, 32)        │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_1_bn (BatchNormalization)    │ (None, 112, 112, 32)        │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_1_relu (ReLU)                │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_1 (Conv2D)                   │ (None, 112, 112, 64)        │           2,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_1_bn (BatchNormalization)    │ (None, 112, 112, 64)        │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_1_relu (ReLU)                │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pad_2 (ZeroPadding2D)           │ (None, 113, 113, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_2 (DepthwiseConv2D)          │ (None, 56, 56, 64)          │             576 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_2_bn (BatchNormalization)    │ (None, 56, 56, 64)          │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_2_relu (ReLU)                │ (None, 56, 56, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_2 (Conv2D)                   │ (None, 56, 56, 128)         │           8,192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_2_bn (BatchNormalization)    │ (None, 56, 56, 128)         │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_2_relu (ReLU)                │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_3 (DepthwiseConv2D)          │ (None, 56, 56, 128)         │           1,152 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_3_bn (BatchNormalization)    │ (None, 56, 56, 128)         │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_3_relu (ReLU)                │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_3 (Conv2D)                   │ (None, 56, 56, 128)         │          16,384 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 3,233,989 (12.34 MB)

 Trainable params: 3,181,445 (12.14 MB)

 Non-trainable params: 52,544 (205.25 KB)

### Training

In [77]:
model.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [78]:
model.fit(x=train_batches, validation_data=valid_batches, epochs=5, verbose=2)

Epoch 1/5
28/28 - 38s - 1s/step - accuracy: 1.0000 - loss: 0.0036 - val_accuracy: 1.0000 - val_loss: 2.8955e-04
Epoch 2/5
28/28 - 17s - 603ms/step - accuracy: 1.0000 - loss: 0.0068 - val_accuracy: 1.0000 - val_loss: 2.3654e-04
Epoch 3/5
28/28 - 20s - 717ms/step - accuracy: 1.0000 - loss: 0.0039 - val_accuracy: 1.0000 - val_loss: 2.0250e-04
Epoch 4/5
28/28 - 18s - 652ms/step - accuracy: 1.0000 - loss: 0.0027 - val_accuracy: 1.0000 - val_loss: 1.8400e-04
Epoch 5/5
28/28 - 18s - 629ms/step - accuracy: 1.0000 - loss: 0.0032 - val_accuracy: 1.0000 - val_loss: 1.3711e-04


### Predictions

#
some useful functions for a bit later

In [63]:
class_names = list(train_batches.class_indices.keys())

# customized 
class_names = ['Captain America (Steve Rogers)', 'Thor (Thor Odinson)', 'Hulk (Bruce Banner)', 'Ironman (Tony Stark)', 'Black Widow (Natasha Romanoff)']
class_names

['Captain America (Steve Rogers)',
 'Thor (Thor Odinson)',
 'Hulk (Bruce Banner)',
 'Ironman (Tony Stark)',
 'Black Widow (Natasha Romanoff)']

In [61]:
def prepare_image(file):
    img_path = './'
    img = image.load_img(img_path + file, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return tf.keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)

def predictImageData(filename):
    preprocessed_image = prepare_image(filename)
    predictions = model.predict(preprocessed_image)
    
    # Extract the first (and only) sample's scores
    scores = predictions[0]
    for i, score in enumerate(scores):
        name = class_names[i]
        percentage = score * 100
        print(f"{name:15}: {percentage:>6.2f}%")

    winner_idx = np.argmax(scores)
    print(f"\nFinal Result: {class_names[winner_idx]} (Confidence: {scores[winner_idx]*100:.2f}%)")


def predictImage(filename):
    preprocessed_image = prepare_image(filename)
    predictions = model.predict(preprocessed_image)
    predicted_index = np.argmax(predictions, axis=1)[0] # get the index of the highest prediction 
    predicted_class = class_names[predicted_index]  # map index to class name
    print("This is: ", predicted_class)

In [81]:
# Automated Prompt 
# requires you be in a test directory, or any directory with images to classify

file_input = input("Filename: ")
print("Short Answer")
print("="*15)
print()
predictImage(f'{file_input}')
print("\n\n")
print("Reasoning: ")
print("="*15)
predictImageData(f'{file_input}')

Short Answer

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
This is:  Black Widow (Natasha Romanoff)



Reasoning: 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
Captain America (Steve Rogers):   5.13%
Thor (Thor Odinson):   0.57%
Hulk (Bruce Banner):  12.62%
Ironman (Tony Stark):  30.76%
Black Widow (Natasha Romanoff):  50.92%

Final Result: Black Widow (Natasha Romanoff) (Confidence: 50.92%)
